In [ ]:
import numpy as np
import imageio
from IPython.display import Image as Img
import IPython.display as display
from PIL import Image

import tensorflow as tf
nn = tf.keras

In [ ]:
IMG_PATH_1 = "/content/dream_input.jpg"

# DeepDream: [Inceptionism-Going Deeper into Neural Networks](https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html)

### Blog
* Instead of exactly prescribing which feature we want the network to amplify, we can also let the network makethat decision. In this case we simply feed the network an arbitrary image or photo and let the network analyze thepicture. We then pick a layer and ask the network to enhance whatever it detected. Each layer of the networkdeals with features at a different level of abstraction, so the complexity of features we generate depends onwhich layer we choose to enhance.
---
* Lower layers tend to produce strokes or simple ornament-likepatterns, because those layers are sensitive to basic features such as edges and their orientations.
---
* If we choose higher-level layers, which identify more sophisticated features in images, complex features or evenwhole objects tend to emerge. Again, we just start with an existing image and give it to our neural net. We ask thenetwork: “Whatever you see there, I want more of it!”
---
* Example: if a cloud looks a little bit like a bird, the network will make it look more like a bird. This in turn will make the network recognize the bird
even more strongly on the next pass and so forth, until a highly detailed bird appears, seemingly out of nowhere.
---
* This technique gives us a qualitative sense of the level of abstraction that a particular layer has achieved in itsunderstanding of images.
---
* If we apply the algorithm iteratively on its own outputs and apply some zooming after each iteration, we get an endless stream of new impressions, exploring the set of things the network knows about. We can even start this process from a random-noise image, so that the result becomes purely the result of the neural network, as seen in the following [***images***](https://photos.google.com/share/AF1QipPX0SCl7OzWilt9LnuQliattX4OUCj_8EP65_cTVnBmS1jnYgsGQAieQUc1VQWdgQ?key=aVBxWjhwSzg2RjJWLWRuVFBBZEN1d205bUdEMnhB)

### Details of DeepDream
* The idea in DeepDream is to choose a layer (or layers) and maximize the "loss" in a way that the image increasingly "excites" the layers.
---
*  For DeepDream, the layers of interest are those where the convolutions are concatenated. There are 11 of these layers in `InceptionV3`, named `'mixed0'` though `'mixed10'`.
---
* Using different layers will result in different dream-like images. Deeper layers respond to higher-level features (such as eyes and faces), while earlier layers respond to simpler features (such as edges, shapes, and textures).
---
* Deeper layers (those with a higher index) will take longer to train on since the gradient computation is deeper.

In [ ]:
base_model = nn.applications.InceptionV3(include_top=False)

In [ ]:
def path_to_image(img_path):
  return Image.open(img_path)

def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)

def show(img):
  display.display(Image.fromarray(np.array(img)))

In [ ]:
show(np.array(path_to_image(IMG_PATH_1))) # (360, 540, 3)

### Loss Function for Dreaming!

* The loss is the sum of the activations in the chosen layers. The loss is normalized at each layer so the contribution from larger layers does not outweigh smaller layers.
---
* We will maximize this loss via gradient ascent.

In [ ]:
class DreamLoss(nn.losses.Loss):
  """Loss for our DeepDream Model"""
  def call(self, image, model):
    image_batch = image[tf.newaxis]
    layer_activations = model(image_batch)
    layer_activations = [layer_activations] if not isinstance(layer_activations, list) else layer_activations

    # taking mean of layer activation to get a single value
    losses = [tf.reduce_mean(activation) for activation in layer_activations] # len(losses) = number of layers selected
    # losses calculated from all output activations are added
    return tf.reduce_sum(losses)

# Gradient Ascent

* Now we calculate the gradients with respect to the image, and add them to the original image
---
* Adding the gradients to the image enhances the patterns seen by the network. At each step, you will have created an image that increasingly excites the activations of certain layers in the network.

In [ ]:
class DeepDream(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.float32),)
  )
  def __call__(self, image, steps, increment):
    loss_fn = DreamLoss()
    for _ in tf.range(steps):
      with tf.GradientTape() as tape:
        tape.watch(image)
        loss = loss_fn(image, self.model)
      gradient = tape.gradient(loss, image)
      gradient /= (tf.math.reduce_std(gradient) + 1e-6)

      image += gradient*increment # loss is maximized to exite the activations in layer output
      image = tf.clip_by_value(image, -1, 1)

    return loss, image

In [ ]:
def train(image, model, steps=150, increment=0.01):
    image = tf.convert_to_tensor(nn.applications.inception_v3.preprocess_input(image))
    increment = tf.convert_to_tensor(increment)
    steps_remaining = steps
    step = 0
    while steps_remaining:
        if steps_remaining>100:
            run_steps = tf.convert_to_tensor(100)
        else:
            run_steps = tf.convert_to_tensor(steps_remaining)
        steps_remaining -= run_steps
        step += run_steps

        loss, image = DeepDream(model=model)(image, run_steps, tf.constant(increment))

        display.clear_output(wait=True)
        show(deprocess(image))
        print (f"Step {step}, loss {loss}")

    final_image = deprocess(image)
    display.clear_output(wait=False)
    show(final_image)
    return final_image

In [ ]:
LAYERS = [layer.name for layer in base_model.layers]
print("Number of layers:", len(LAYERS))
layer_out = [base_model.get_layer(name).output for name in LAYERS]

In [ ]:
def dreamed_image(image_path, layer_out):
    image = np.array(path_to_image(image_path))
    model = nn.Model(inputs=base_model.input, outputs=layer_out)
    out_img = train(image, model, steps=200, increment=0.01)
    return out_img

In [ ]:
def make_gif(path_to_save):
    img_list = []
    for i in range(len(LAYERS)):
        out_img = dreamed_image(IMG_PATH_1, layer_out=layer_out[i])
        img_list.append(out_img)
    imageio.mimsave(path_to_save, tf.convert_to_tensor(img_list).numpy(), format='GIF', duration=2)
    return

In [ ]:
path = "inception_200_001.gif"
make_gif(path)
Img(filename=path)